In [ ]:
import random
from models import deck
from scipy.stats import multivariate_hypergeom, hypergeom, nhypergeom
from matplotlib import pyplot

Specify the filesystem path in `filepath` variable, then load the deck into memory

In [ ]:
filepath = "./deck/madolche.txt"
mydeck = deck.Deck.from_file(filepath)

Validate the total cards of each type:

In [ ]:
def total_count(cards):
    return sum([i.count for i in cards])


bricks = mydeck.cards_by_deck_type(deck.DeckCardType.BRICK)
non_engine = mydeck.cards_by_deck_type(deck.DeckCardType.NONENGINE)
engine = mydeck.cards_by_deck_type(deck.DeckCardType.ENGINE)

print(
    f"Your deck has {total_count(bricks)} bricks, {total_count(non_engine)} non-engine, and {total_count(engine)} engine out of {mydeck.size} cards"
)

Chance to see any 1 card combo:

In [ ]:
one_card = list(filter(lambda x: len(x.cards) == 1, mydeck.combos))
one_card_cards = set([card for combo in one_card for card in combo.cards])
total_one_card = sum([i.count for combo in one_card for i in combo.cards])
one_card_pv = nhypergeom.pmf(0, mydeck.size, total_one_card, 5)
print(f"All 1 card combos: {one_card_cards}")
print(f"You'll see a 1 card combo in {(1-one_card_pv)*100}% of hands")

Chance to see any 2 card combo:
1. Find all discrete cards in all the combos
2. Calculate the probability that the hand has 0 of the cards in the combo
3. For each combo, calculate that only one of the cards is present in the hand

In [ ]:
two_card = list(filter(lambda x: len(x.cards) == 2, mydeck.combos))
cards_in_two_card = set([card for combo in two_card for card in combo.cards])
# Find hands where none of the cards from the combo are in it at all
total_no_card = sum([i.count for i in cards_in_two_card])
no_cards = hypergeom.cdf(0, mydeck.size, total_no_card, 5)
# For each card in 2 card combos, calculate exactly once the probablility of seeing one
# card from the combo and not the other.
combos_calculated = {}
for card in cards_in_two_card:
    # Find all combos with this card
    combos = list(filter(lambda x: card in x.cards, two_card))
    for combo in combos:
        if combo in combos_calculated.keys():
            continue
        total = sum([i.count for i in combo.cards])
        total_card = combo.cards[0].count
        total_other_card = combo.cards[1].count
        rv = multivariate_hypergeom(
            m=[
                total_card,
                total_other_card,
                mydeck.size - total_other_card - total_card,
            ],
            n=5,
        )
        p_1 = sum([rv.pmf([x, 0, 5 - x]) for x in range(1, total_card)])
        p_2 = sum([rv.pmf([0, x, 5 - x]) for x in range(1, total_other_card)])
        p_0 = rv.pmf([0, 0, 5])
        combos_calculated[combo] = 1 - (p_1 + p_2 + p_0)
for combo, prob in combos_calculated.items():
    print(f"You'll see {combo} in {prob*100}% of hands")

Chance to brick with half of a 2 card combo, and no one card combos:

In [ ]:
# Find all cards in combos
cumulative_probability = 1
cards_in_combos = set([card for combo in mydeck.combos for card in combo.cards])
total = sum([i.count for i in cards_in_combos])
chance_to_see_zero = hypergeom.pmf(0, mydeck.size, total, 5)
cumulative_probability = cumulative_probability - chance_to_see_zero
print(f"Chance to see 0 combo cards: {chance_to_see_zero*100}%")
# Find 2 card combos that do not include 1 card combos
unique_two_card = list(
    filter(lambda x: len(one_card_cards.intersection(set(x.cards))) == 0, mydeck.combos)
)
# Get all unique cards in 2 card combos, calculate the odds of see it, but not the counterpart
cards_in_unique_two_card = set(
    [card for combo in unique_two_card for card in combo.cards]
)
for card in cards_in_unique_two_card:
    # Find combos for card
    combos = list(filter(lambda x: card in x.cards, unique_two_card))
    if len(combos) == 1:
        continue
    other_cards = set([card for combo in combos for card in combo.cards]).difference(
        [card]
    )
    total_other_card = sum([i.count for i in other_cards.union(one_card_cards)])
    rv = multivariate_hypergeom(
        m=[card.count, total_other_card, mydeck.size - total_other_card - card.count],
        n=5,
    )
    p = sum([rv.pmf([x, 0, 5 - x]) for x in range(1, card.count + 1)])
    print(f"Chance to see hands with {card.name}, but nothing needed to play: {p*100}%")

Run a simulation. Input the number of hands to draw in `N`, then see how many have a one-card or two card combo: 

In [ ]:
N = 100
hands = [tuple(random.sample(mydeck.get_simulated_cards(), 5)) for _ in range(0, N)]
hands_with_combo = set()
for combo in mydeck.combos:
    in_hands = []
    cards: set = set(combo.cards)
    for hand in hands:
        if cards.issubset(hand):
            in_hands.append(hand)
            hands_with_combo.add(hand)
    print(f"Combo {combo} in {len(in_hands)} out of {len(hands)}")
bricks = set(hands).difference(hands_with_combo)

In [ ]:
print(f"{len(bricks)} hands are bricks:")

In [ ]:
for brick in bricks:
    print(brick)

Hands that had any number of distinct non-engine:

In [ ]:
x = [0, 1, 2, 3, 4, 5]
y = [0, 0, 0, 0, 0, 0]
for hand in hands:
    unique_cards = set(hand)
    ne = list(
        filter(lambda x: x.deck_type == deck.DeckCardType.NONENGINE, unique_cards)
    )
    idx = len(ne)
    y[idx] += 1
pyplot.plot(x, y)